In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import warnings

In [4]:
df = pd.read_csv('data/stud.csv')


In [5]:
X = df.drop(columns=['math_score'],axis=1)
y = df['math_score']


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()


preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [9]:
X = preprocessor.fit_transform(X)

In [59]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=45)
X_train.shape, X_test.shape

((800, 19), (200, 19))

In [60]:
def eval_model(true,predicted):
    mae = mean_absolute_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    return (mae,rmse)


In [68]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "AdaBoost Regressor": AdaBoostRegressor()
}


dict_values([LinearRegression(), Lasso(), Ridge(), DecisionTreeRegressor(), RandomForestRegressor(), XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...), AdaBoostRegressor()])

In [62]:
model_list = []
rmse_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Predictions for train
    y_train_pred = model.predict(X_train)
    # Predictions for test
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, = eval_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse,  = eval_model(y_test, y_test_pred)


    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    rmse_list.append(model_test_rmse)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3121
- Mean Absolute Error: 4.2156
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4142
- Mean Absolute Error: 4.3716


Lasso
Model performance for Training set
- Root Mean Squared Error: 6.5511
- Mean Absolute Error: 5.1433
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.4711
- Mean Absolute Error: 5.2436


Ridge
Model performance for Training set
- Root Mean Squared Error: 5.3124
- Mean Absolute Error: 4.2142
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.4126
- Mean Absolute Error: 4.3718


Decision Tree
Model performance for Training set
- Root Mean Squared Error: 0.3750
- Mean Absolute Error: 0.0262
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 8.6026
- Mean Absolute Error: 7.0250


Random Forest Regressor
Model pe

In [63]:
""" print(list(zip(model_list, rmse_list))) will give you:
[('Linear Regression', 5.414248341544381), 
        ('Lasso', 6.471076130248417), 
        ('Ridge', 5.4125626279451655), 
        ('Decision Tree', 8.602615881230545), 
        ('Random Forest Regressor', 6.237944661560765), 
        ('XGBRegressor', 6.812629437454282), 
        ('AdaBoost Regressor', 6.060625840686908)]
"""
pd.DataFrame(list(zip(model_list, rmse_list)), columns=['Model Name', 'rmse_list']).sort_values(by=["rmse_list"],ascending=True)


,Model Name,rmse_list
2,Ridge,5.412563
0,Linear Regression,5.414248
6,AdaBoost Regressor,6.060626
4,Random Forest Regressor,6.237945
1,Lasso,6.471076
5,XGBRegressor,6.812629
3,Decision Tree,8.602616
